In [4]:
from pytube import YouTube

import re
import subprocess
import time
import pandas as pd
import numpy as np
import pickle
import joblib
import whisper

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [27]:
class ALL_Model:
    def __init__(self, url):
        """
        argument: Youtube URL
        """
        self.url = url
        self.yt = YouTube(self.url)
        self.video_url = self.yt.streams.get_highest_resolution().url
        self.audio_url = self.yt.streams.filter(only_audio=True).first().url
        self.easyocr_model = joblib.load("easyocr_model.pkl")
        self.whisper_model = joblib.load("whisper_model.pkl")
        self.caption_df = pd.DataFrame(columns=['start', 'end', 'text'])
        self.easyocr_results = pd.DataFrame(columns=["bbox", "text", "conf", "time"])
        self.whisper_results = pd.DataFrame(columns=["start", "end", "text"])
        self.fps = None
        self.frame_count = None
    
    def get_captions(self):
        """
        YouTube에서 제공하는 자막 정보 받기
        return: pd.DataFrame
        """
        # 첫번째 자막 선택
        caption = self.yt.captions.all()[0]
        
        # YouTube에서 제공하는 자막 DataFrame으로 저장
        for data in caption.json_captions['events']:
            if 'aAppend' in data:
                continue
            if 'segs' in data:
                start = data['tStartMs'] / 1000.0
                end = start + (data['dDurationMs'] / 1000.0)
                text = []
                for seg in data['segs']:
                    text.append(seg['utf8'])
                text = ' '.join(text)
                temp = pd.DataFrame([[start, end, text]], columns=["start", "end", "text"])
                self.caption_df = pd.concat([self.caption_df, temp], ignore_index=True)
        return self.caption_df
    
    def get_easyocr_result(self):
        """
        easyocr을 통해 동영상에서 Text추출
        return: pd.DataFrame
        """
        # fps 추출
        ffmpeg_cmd = ['ffmpeg', '-i', self.video_url, '-f', 'null', '-']
        output = subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT).decode()

        fps_match = re.search(r"\b(\d+\.?\d*)\sfps\b", output)
        self.fps = float(fps_match.group(1)) if fps_match else None
        
        # frame_count 추출
        command = ['ffprobe', '-v', 'error', '-count_frames', '-select_streams', 'v:0', '-show_entries',
               'stream=nb_read_frames', '-of', 'default=nokey=1:noprint_wrappers=1', self.video_url]
        output = subprocess.check_output(command).decode().strip()
        self.frame_count = int(output) if output.isdigit() else None
        
        print("fps :", self.fps)
        print("frame_count :", self.frame_count)
        
        
        # 이미지의 width, height 구해야함.
        wh_command = ['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=width,height',
               '-of', 'csv=s=x:p=0', self.video_url]
        output = subprocess.check_output(wh_command).decode().strip()
        width, height = map(int, output.split('x'))
            
        ffmpeg_a = ['ffmpeg', '-i', self.video_url, '-vf', 'select=not(mod(n\\,{0}))'.format(self.fps), '-vsync', 'vfr',
                  '-f', 'image2pipe', '-pix_fmt', 'rgb24', '-vcodec', 'rawvideo', '-']
            
        # ffmpeg pipeline구성
        ffmpeg_pipe = subprocess.Popen(ffmpeg_a, stdout=subprocess.PIPE, bufsize=10**8)
        
        frames = []
        frame_pos = 0
        #for i in tqdm(range(self.frame_count)):
        while True:
            # 파이프라인에서 프레임 데이터 읽기
            raw_frame = ffmpeg_pipe.stdout.read(height * width * 3)
            if not raw_frame:
                break
            
            frame = np.frombuffer(raw_frame, np.uint8).reshape((height, width, 3))
            frames.append(frame)
            
            result = pd.DataFrame(self.easyocr_model.readtext(frame, detail=1), columns=[
                'bbox', 'text', 'conf'
            ])
            result['time'] = frame_pos / self.fps
            self.easyocr_results = pd.concat([self.easyocr_results, result], ignore_index=True)
            frame_pos += 1
        
        ffmpeg_pipe.stdout.close()
        ffmpeg_pipe.wait()
        
        # 시간을 재기 위해,어쩔수 없이 result를 while문에 구성하였음.
        # while문 빼고, 아랫 하단 for문에서 돌리면 시간이 반으로 줄어듬.(이유: 배치파일, 병렬처리 때문인것 같음)
        # for i in tqdm(range(len(frames))):  
            # result = pd.DataFrame(self.easyocr_model.readtext(frame, detail=1), columns=[
            #     'bbox', 'text', 'conf'
            # ])
            # # result['time'] = frame_pos / self.fps
            
            # self.easyocr_results = pd.concat([self.easyocr_results, result], ignore_index=True)
            # frame_pos += 1
        
        return self.easyocr_results
    
    def get_whisper_result(self):
        """
        Whisper를 통해 동영상 오디오에서 Text 추출
        return: pd.DataFrame
        """
        audio_all = whisper.load_audio(self.audio_url) # load audio
        result = self.whisper_model.transcribe(audio_all)

        for seg in result['segments']:
            start, end, text = seg['start'], seg['end'], seg['text']
            print(f"[ {start:>6.2f} ~ {end:>6.2f} ] {text}")
            temp = pd.DataFrame([[start, end, text]], columns=["start", "end", "text"])
            self.whisper_results = pd.concat([self.whisper_results, temp], ignore_index=True)
        return self.whisper_results
    

if __name__ == "__main__":
    url = 'https://www.youtube.com/watch?v=bGcVkNP1tPs&t=2s&ab_channel=1%EB%B6%84%EB%AF%B8%EB%A7%8C'
    models = ALL_Model(url=url)
    caption = models.get_captions()
    caption.to_csv("caption.csv")
    easyocr_result = models.get_easyocr_result()
    easyocr_result.to_csv("easyocr_result.csv")
    whisper_result = models.get_whisper_result()
    whisper_result.to_csv("whisper_result.csv")

C:\Users\minsu\AppData\Local\Temp\ipykernel_33052\2909942494.py:24: DeprecationWarning: Call to deprecated function all (This object can be treated as a dictionary).
  caption = self.yt.captions.all()[0]


fps : 30.0
frame_count : 3594


c:\Users\minsu\anaconda3\envs\deeplearning\lib\site-packages\easyocr\utils.py:576: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = cv2.resize(img,(int(model_height*ratio),model_height),interpolation=Image.ANTIALIAS)
c:\Users\minsu\anaconda3\envs\deeplearning\lib\site-packages\easyocr\recognition.py:93: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resized_image = image.resize((resized_w, self.imgH), Image.BICUBIC)
c:\Users\minsu\anaconda3\envs\deeplearning\lib\site-packages\easyocr\utils.py:576: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = cv2.resize(img,(int(model_height*ratio),model_height),interpolation=Image.ANTIALIAS)
c:\Users\minsu\anaconda3\envs\deeplearning\lib\site-packages\easyocr\recognition.py:93: DeprecationWarning: BICUBIC is depr

[   0.00 ~   4.00 ]  자 오늘은 지금 전국민이 혼란에 빠진 우회전 관련 최종정리 영상인데요.
[   4.00 ~   7.04 ]  이거 뉴스에서도 정작 중요한 건 안 알려져서 거의 깔리게 만드는
[   7.04 ~   8.60 ]  시간엽수니 핵심한 빠르게가 부족.
[   8.60 ~  12.68 ]  일단 첫 번째 사실 지금 단속되고 있는 차량 90%는 다 이것 때문인데요.
[  12.68 ~  15.48 ]  바로 도대체 어디서 일시 정지를 해야 되냐는 겁니다.
[  15.48 ~  21.08 ]  결론부터 말씀드리면 대부분 여기 우회전에서 만나는 보행자 신호등에서 멈춰야 되는 걸로 알고 있는데요.
[  21.08 ~  21.68 ]  아닙니다.
[  21.68 ~  26.44 ]  정답은 바로 여기 우회전하기 전에 만나는 횡단보도에 있는 정지선에서 일시 정지해야 돼요.
[  26.44 ~  28.64 ]  확실하게 경찰청 공식 입장을 들어보죠.
[  28.64 ~  33.44 ]  앞에 운영이 진행하고자 하는 방향에 적식 적신호가 들어왔을 때는
[  33.44 ~  36.00 ]  처음 만나는 횡단보도 앞에 정지선이 있거든요.
[  36.00 ~  42.56 ]  거기에서 일단 정지해서 자후에 부행자가 있는지 없는지 확인하고 그 다음에 진행을 해야 되는 상황이고요.
[  42.56 ~  47.32 ]  네, 한마디로 전방 신호등이 빨간 부릴때 바로 앞 정지선에 맞춰서 일시 정지를 해야 된다는 겁니다.
[  47.32 ~  50.52 ]  그리고 녹색 부릴때는 지금까지와 똑같이 서행에서 우회전을 하던
[  50.52 ~  55.36 ]  부행자 유무에 따라서 천천히 지나갈 수도 있고 다시 한번 멈췄다가 가야 될 수도 있는 거죠.
[  55.36 ~  56.08 ]  바로 두 번째.
[  56.08 ~  58.40 ]  그럼 도대체 일시 정지가 몇초냐는 건데요.
[  58.40 ~  61.76 ]  지금 인터넷에 보면 3초다 5초다 말들이 많